In [1]:
import numpy as np 
import math
import pandas as pd 
import pandas_datareader as pdr 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import sequential
from keras.layers import Dense , LSTM
from keras.models import Sequential
plt.style.use("fivethirtyeight")
import datetime
from datetime import  timedelta


steps_size = 45

#'HEROMOTOCO.NS','TECHM.NS','COALINDIA.NS','ITC.NS','LTI.NS','INDUSINDBK.NS','AMARAJABAT.NS','BERGEPAINT.NS','GUJGASLTD.NS','HDFC.NS','HDFCAMC.NS','HDFCBANK.NS', 'DABUR.NS','DEEPAKNTR.NS','INFY.NS','MIDHANI.NS','BRITANNIA.NS','LTTS.NS','WIPRO.NS','TCS.NS','AARTIIND.NS',
set = ['COLPAL.NS','BHARTIARTL.NS','ASHOKLEY.NS','MARICO.NS','KOTAKBANK.NS','ICICIBANK.NS','IRCON.NS','TATAELXSI.NS','RVNL.NS','MGL.NS','HINDUNILVR.NS','IGL.NS','IRFC.NS','PETRONET.NS','NTPC.NS','HEROMOTOCO.NS','TECHM.NS','COALINDIA.NS','HINDALCO.NS']
for i in range(len(set)):



   
    # Get the stock quote
    # name should be exact from yahoo finance site
    stock_name = set[i]
    df = pdr.DataReader(stock_name , data_source='yahoo',start='2000-01-01',end=datetime.datetime.now())

    # show the data 
    df
    
    print("stock name : " , stock_name )

    #df.to_csv('price.csv') to save data as csv

    print(df.shape)

    # Creating a new df for only close price 
    #data = df.filter(['Close','High'])           # gives 2 column 
    data = df.filter(['Close'])                   # gives 1 column
    data

    # create df to a  num py array         df.values : Only the values in the DataFrame will be returned, the axes labels will be removed.

    dataset = data.values
    dataset


    # Get the no of adta to train the model on say 80%
    training_data_len = math.ceil(len(dataset)*0.80)                    #Return the ceiling of x as an Integral.
    training_data_len

    # scale the data 
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(dataset)                        # range [0-1] on numpy array of dataset
    scaled_data




    # create a scaled training data set 
    train_data = scaled_data[0:training_data_len , : ]                # till row = training data len all column points

    # split the data into x_train  , and y_train data sets
    x_train = []                                                      # features = independent variables 
    y_train = []                                                      # lables = output = dependent variables 

    for i in range(steps_size , len(train_data)): 
        x_train.append(train_data[i-steps_size:i , 0])                        # we are going to append 0 to 89th  values of train_data in 0th column
        y_train.append(train_data[i , 0])                             # lable/ prediction is 90th data point of 0th column from train_data to y_train

        if i <= (steps_size+2):
            print(x_train)
            print()
            print(y_train)




    # convert x_train and y_train to numpy arrays so that array data can be provided to LSTM model
    x_train , y_train = np.array(x_train) , np.array(y_train)

    # x_train.shape               (2363, 30)   # after appending in every row tere are 30 columns of data 

    # x_train                                  # is 2d type row  and column data

    # reshaping the x_train data set as a LSTM model expects the input to be a three dimentionals array 
    x_train = np.reshape(x_train ,newshape= (x_train.shape[0] , x_train.shape[1] , 1))
    x_train.shape

    # Build the LSTM model 
    model = Sequential()

    # add a layer of 50 neuron , there output to be used in next layer thus return seq = true , since first layer thus shape of input neuron   , 
    # input shape = (time step , features ) = (90 days , 1 ) = (90 , close price)
    model.add(LSTM(50, activation ='relu', return_sequences=True, input_shape = (x_train.shape[1],1)))

    # add 1 last lstm layer but no lstm further thus return_sequence = False
    model.add(LSTM(50, return_sequences=False))

    # adding last dense layer          
    model.add(Dense(25))  

    # adding last dense layer          
    model.add(Dense(1)) 


    model.summary()



    # compile the model 
    # optimizer is used to minimise the loss fx 
    # loss fx is used to calculate what was the loss to see how well the model did on traiing 

    model.compile(optimizer='adam' , loss="mean_squared_error")




    # train the model
    model.fit(x=x_train,y=y_train,batch_size=1,epochs=10)



    model.save('{}_{}_keras_model.h5'.format(stock_name, steps_size))
    
    # creating the testing data set
    # create a new array containing scaled values from index  end of len(train_data) till end of the total data
    
    from keras.models import load_model 
    model = load_model('{}_{}_keras_model.h5'.format(stock_name, steps_size))
    
    test_data = scaled_data[training_data_len - steps_size : , : ]         # all data from 60th to end of data with all column

    # creating a dataset x_test and y_test
    x_test = []
    y_test = dataset[training_data_len : , : ]                          # actual prediction values
                                                                        # dataset values are original closing prce and not scaled

    for i in range(steps_size , len(test_data)):
        x_test.append(test_data[i-steps_size : i, 0 ])



    # create the data to a numpy array as model need 3d array
    x_test = np.array(x_test)

    # reshape the data for the model      (newshape =no of rows = x_test.shape[0], timesteps = 60 days = x_test.shape[1] , and 1 )
    x_test = np.reshape(x_test , newshape= (x_test.shape[0] , x_test.shape[1] , 1 ))


    y_predictions = model.predict(x_test)    # scaled value




    # undo scaling
                                                               # we want the predictions to be exact same as real lables 
    y_predictions = scaler.inverse_transform(y_predictions)        # Undo the scaling of X according to feature_range(0-1)
                                                               # gaining back the real value

    # EVALUATE THE MODEL root mean square error (RMSE)
    # its a measure of how good the model predicts the response 
    # can be compared by std deviation less
    # lower value of rmse is better : try diff range of dataset 

    rmse = np.sqrt(np.mean(y_predictions - y_test)**2)
    print('Root mean square error in predicted vs actual values ', rmse)
    print("stock name " , stock_name )
    

stock name :  COLPAL.NS
(4929, 6)
[array([0.0085973 , 0.00818314, 0.00821171, 0.00806889, 0.00806889,
       0.00896861, 0.00848305, 0.00824026, 0.00804033, 0.00825455,
       0.00945417, 0.00895433, 0.00841164, 0.00888292, 0.00854017,
       0.00841164, 0.00842592, 0.00832595, 0.00832595, 0.00799749,
       0.00734055, 0.00655508, 0.0065408 , 0.00686927, 0.00668361,
       0.00686927, 0.00661221, 0.00648368, 0.00655508, 0.00685499,
       0.00678358, 0.00695495, 0.00668361, 0.00668361, 0.00681214,
       0.00691211, 0.00684071, 0.00668361, 0.00645511, 0.00669789,
       0.00675502, 0.00741196, 0.00762617, 0.00725486, 0.00736911])]

[0.007326267354345259]
[array([0.0085973 , 0.00818314, 0.00821171, 0.00806889, 0.00806889,
       0.00896861, 0.00848305, 0.00824026, 0.00804033, 0.00825455,
       0.00945417, 0.00895433, 0.00841164, 0.00888292, 0.00854017,
       0.00841164, 0.00842592, 0.00832595, 0.00832595, 0.00799749,
       0.00734055, 0.00655508, 0.0065408 , 0.00686927, 0.00668361,
